influential_interpretation_nn.ipynb## load data

In [1]:
import sys
sys.path.append('../../')

In [2]:
import json
import pandas as pd
from mlinterpreter.util import preprocess_data

In [3]:
# 读入数据 和 schema
data_path = "../../demo_data/adult.data"
schema_path = "../../demo_data/schema.json"
df = pd.read_csv(data_path, header=None)
df.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex",
                 "capital-gain","capital-loss","hours-per-week","native-country","income"]
with open(schema_path, "r") as f:
    schema = json.load(f)

## preprocess data

In [4]:
# 预处理数据
preprocess_df, preprocess_dict = preprocess_data(df=df, schema=schema, process_label=True)

## influential instances

获取最有影响样本
    
算法说明：
   - 将数据集切分成 train 和 valid
   - 在train上训练NN模型
   - 计算将train中某个样本剔除后，valid上loss的变化大小
       - 这里都是用近似估计的方式计算valid上loss的变化
 
 
算法(get_influential_instances_nn_sgd)
[Data Cleansing for Models Trained with SGD.(NeurIPS'19]
   - 通过一阶泰勒展开和类似反向梯度传播的方法计算train样本对valid loss的影响，适用于基于SGD的训练方法

In [5]:
from mlinterpreter.instance.influence import get_influential_instances_nn_sgd

In [6]:
top10_influential_instances, influence = get_influential_instances_nn_sgd(
    preprocess_df[:500], 
    df_display=df[:500], 
    schema=schema, 
    split=0.2, 
    topk=10)

Epoch 1/10
2/2 [==============================] - 4s 854ms/step - loss: 0.6911 - auc: 0.4948 - val_loss: 0.6514 - val_auc: 0.5178
Epoch 2/10
2/2 [==============================] - 0s 64ms/step - loss: 0.6408 - auc: 0.5179 - val_loss: 0.6054 - val_auc: 0.5778
Epoch 3/10
2/2 [==============================] - 0s 44ms/step - loss: 0.6048 - auc: 0.5494 - val_loss: 0.5703 - val_auc: 0.6181
Epoch 4/10
2/2 [==============================] - 0s 47ms/step - loss: 0.5773 - auc: 0.5813 - val_loss: 0.5427 - val_auc: 0.6646
Epoch 5/10
2/2 [==============================] - 0s 48ms/step - loss: 0.5558 - auc: 0.6157 - val_loss: 0.5210 - val_auc: 0.7098
Epoch 6/10
2/2 [==============================] - 0s 63ms/step - loss: 0.5389 - auc: 0.6526 - val_loss: 0.5040 - val_auc: 0.7495
Epoch 7/10
2/2 [==============================] - 0s 45ms/step - loss: 0.5258 - auc: 0.6848 - val_loss: 0.4907 - val_auc: 0.7734
Epoch 8/10
2/2 [==============================] - 0s 43ms/step - loss: 0.5152 - auc: 0.7064 - va

100%|██████████| 20/20 [03:16<00:00,  9.81s/it]


In [7]:
print(top10_influential_instances)

     age          workclass  fnlwgt      education  education-num  \
387   44            Private  162028   Some-college             10   
183   40            Private  286370        7th-8th              4   
214   54       Self-emp-inc  125417        7th-8th              4   
469   30            Private  118551      Bachelors             13   
189   58          State-gov  109567      Doctorate             16   
215   37            Private  635913      Bachelors             13   
433   30            Private   29662     Assoc-acdm             12   
37    19            Private  544091        HS-grad              9   
80    18            Private  446839        HS-grad              9   
272   50   Self-emp-not-inc   30653        Masters             14   

          marital-status          occupation    relationship    race      sex  \
387   Married-civ-spouse        Adm-clerical            Wife   White   Female   
183   Married-civ-spouse   Machine-op-inspct         Husband   White     Male 